In [27]:
import pandas as pd
import random
import time
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import ActionChains

In [28]:
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
path = f'https://scholar.google.com/scholar?start=0&q=source:CHI+source:conference+source:on+source:human+source:factors+source:in+source:computing+source:systems&hl=en&as_sdt=0,5&as_ylo=2023&as_yhi=2023&as_vis=1'
driver.get(path)

# path = f'https://scholar.google.com/scholar?as_q=&as_epq=&as_oq=&as_eq=&as_occt=any&as_sauthors=&as_publication=IEEE+Transactions+on+Pattern+Analysis+and+Machine+Intelligence+&as_ylo=&as_yhi=&hl=en&as_sdt=0%2C5'

In [29]:
columns = ['title', 'web', 'cite', 'pdf', 'abstract']
CHI_2023 = pd.DataFrame(columns=columns)
article_num = 0

In [32]:
while True:

    # 固定时间保存一下，以防不测
    if article_num % 100 == 0:
        CHI_2023.to_csv('./output/CHI_2023.csv', index=False)

    all_article_contents = driver.find_elements(By.CLASS_NAME, 'gs_r.gs_or.gs_scl')

    for all_article_content in all_article_contents:

        pdf_link = ''
        web_link = ''
        title = ''
        cite = 0

        print(f'>>>>>>>>>>>>>>>>>>    Paper {article_num}     <<<<<<<<<<<<<<<<<<<')

        article_content = all_article_content.find_element(By.CLASS_NAME, 'gs_ri')

        # 存储web_link和title
        title_link_content = article_content.find_element(By.XPATH, './/h3/a')
        title = title_link_content.text
        print('Title:', title)
        web_link = title_link_content.get_attribute('href')
        print('Web link:', web_link)

        # 存储引用量
        cite_content = article_content.find_element(By.CLASS_NAME, 'gs_fl.gs_flb')
        cite_str = cite_content.find_element(By.XPATH, './/a[3]').text

        if 'by' in cite_str:
            str_list = cite_str.split(' ')
            cite = int(str_list[2])
        else:
            cite = 0

            # 存储pdf link
        try:
            pdf_content = all_article_content.find_element(By.CLASS_NAME, 'gs_ggs.gs_fl')
            # '.'表示当前节点
            pdf_link_content = pdf_content.find_element(By.XPATH, './/div/div/a')
            pdf_link = pdf_link_content.get_attribute('href')
            print('PDF link:', pdf_link)
        except:
            print('No pdf !!!')

        print("Cite:", cite)

        CHI_2023.loc[article_num, 'title'] = title
        CHI_2023.loc[article_num, 'web'] = web_link
        CHI_2023.loc[article_num, 'pdf'] = pdf_link
        CHI_2023.loc[article_num, 'cite'] = cite
        article_num += 1

        print()

    # 在当前位置滚动指定像素
    delta_y = 1200
    ActionChains(driver).scroll_by_amount(0, delta_y).perform()
    mouse_tracker = driver.find_element(By.ID, "mouse-tracker")
    ActionChains(driver).move_to_element_with_offset(mouse_tracker, 8, 11).perform()

    try:
        clickable = driver.find_element(By.XPATH, '//*[@id="gs_n"]/center/table/tbody/tr/td[12]/a/b')
        time.sleep(1 + 2 * random.random())
        ActionChains(driver).click(clickable).perform()
    except:
        print('The Final Page ！！！')
        CHI_2023.to_csv('./output/CHI_2023.csv', index=False)
        break

>>>>>>>>>>>>>>>>>>    Paper 0     <<<<<<<<<<<<<<<<<<<
Title: Co-Writing Screenplays and Theatre Scripts with Language Models: Evaluation by Industry Professionals
Web link: https://dl.acm.org/doi/abs/10.1145/3544548.3581225
PDF link: https://dl.acm.org/doi/pdf/10.1145/3544548.3581225
Cite: 34

>>>>>>>>>>>>>>>>>>    Paper 1     <<<<<<<<<<<<<<<<<<<
Title: Co-writing with opinionated language models affects users' views
Web link: https://dl.acm.org/doi/abs/10.1145/3544548.3581196
PDF link: https://dl.acm.org/doi/pdf/10.1145/3544548.3581196
Cite: 16

>>>>>>>>>>>>>>>>>>    Paper 2     <<<<<<<<<<<<<<<<<<<
Title: Why Johnny can't prompt: how non-AI experts try (and fail) to design LLM prompts
Web link: https://dl.acm.org/doi/abs/10.1145/3544548.3581388
PDF link: https://dl.acm.org/doi/pdf/10.1145/3544548.3581388
Cite: 16

>>>>>>>>>>>>>>>>>>    Paper 3     <<<<<<<<<<<<<<<<<<<
Title: Studying the effect of AI Code Generators on Supporting Novice Learners in Introductory Programming
Web link: ht

In [34]:
CHI_2023.to_csv('./output/CHI_2023.csv', index=False, encoding='utf-8-sig')